In [0]:
df_silver = spark.readStream.table("prod.silver.orders_clean")

df_gold = (
    df_silver
        .groupBy(window(col("event_time"), "1 day"))
        .agg(
            sum("order_amount").alias("total_sales"),
            count("order_id").alias("total_orders")
        )
)

(
    df_gold.writeStream
        .format("delta")
        .option("checkpointLocation", "abfss://checkpoints/gold/daily_sales")
        .outputMode("complete")
        .toTable("prod.gold.daily_sales")
)